In [1]:
import import_ipynb
from maze_common import *
import time
import random as rnd
import numpy as np
import copy

importing Jupyter notebook from maze_common.ipynb


In [2]:
maze = Maze(40, .30)
solvable = maze.isSolvable()
print("Is solvable?", solvable)
print("Obstacles?", maze.obstacles.shape[0])

Is solvable? True
Obstacles? 485


In [3]:
if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 1043
Shortest path length: 79 
Shortest path: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 5), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (8, 10), (9, 10), (10, 10), (11, 10), (11, 11), (12, 11), (12, 12), (12, 13), (13, 13), (13, 14), (14, 14), (15, 14), (16, 14), (17, 14), (18, 14), (18, 15), (19, 15), (20, 15), (20, 16), (20, 17), (20, 18), (20, 19), (21, 19), (21, 20), (21, 21), (21, 22), (22, 22), (23, 22), (23, 23), (24, 23), (25, 23), (26, 23), (26, 24), (26, 25), (27, 25), (28, 25), (29, 25), (30, 25), (31, 25), (31, 26), (31, 27), (32, 27), (32, 28), (33, 28), (34, 28), (35, 28), (35, 29), (36, 29), (36, 30), (36, 31), (37, 31), (37, 32), (37, 33), (38, 33), (38, 34), (38, 35), (38, 36), (38, 37), (39, 37), (39, 38), (39, 39)]


In [5]:
def manhattanDistance(coords, maze):
    cellRow, cellCol = coords
    return ((maze.dim - 1) - cellRow) + ((maze.dim - 1) - cellCol), 0

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 284
Shortest path length: 79 
Shortest path: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (8, 2), (9, 2), (10, 2), (10, 3), (10, 4), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (12, 10), (12, 11), (12, 12), (12, 13), (13, 13), (13, 14), (14, 14), (15, 14), (16, 14), (17, 14), (18, 14), (18, 15), (19, 15), (20, 15), (20, 16), (20, 17), (20, 18), (20, 19), (20, 20), (21, 20), (21, 21), (21, 22), (22, 22), (23, 22), (24, 22), (25, 22), (26, 22), (26, 23), (26, 24), (27, 24), (28, 24), (28, 25), (29, 25), (30, 25), (31, 25), (31, 26), (31, 27), (32, 27), (33, 27), (33, 28), (34, 28), (35, 28), (35, 29), (36, 29), (36, 30), (36, 31), (37, 31), (37, 32), (37, 33), (38, 33), (38, 34), (38, 35), (39, 35), (39, 36), (39, 37), (39, 38), (39, 39)]


In [6]:
def thinMaze(maze, fractionRemove):
    thinMaze = Maze(1, 0)
    thinMaze.board = copy.deepcopy(maze.board)
    thinMaze.obstacles = copy.deepcopy(maze.obstacles)
    thinMaze.dim = maze.dim
    numRemove = round(fractionRemove * thinMaze.obstacles.shape[0])
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX, obstacleY] = Cell.OPEN
    return thinMaze
    
thinnedMaze = thinMaze(maze, .5)
print("Original obstacles: ", maze.obstacles.shape[0])
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0])
# print("Thinned maze:\n", thinnedMaze.board)

Original obstacles:  485
Thinned obstacles:  243


In [7]:
visited = {}

def thinnedMazeShortestPathLength(coords, maze):
    cellRow, cellCol = coords
    shortestPathLength = 0
    expandedCells = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, expandedCellsBySearch = shortestPathSearch(thinnedMaze, startCoords = (cellRow, cellCol))
        shortestPathLength = len(shortestPath)
        expandedCells = expandedCellsBySearch
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength, expandedCells

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Expanded cells: 87120
Shortest path length: 79 
Shortest path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (11, 8), (11, 9), (12, 9), (12, 10), (12, 11), (12, 12), (12, 13), (12, 14), (13, 14), (14, 14), (15, 14), (16, 14), (17, 14), (17, 15), (17, 16), (17, 17), (17, 18), (18, 18), (19, 18), (19, 19), (19, 20), (20, 20), (21, 20), (21, 21), (21, 22), (22, 22), (23, 22), (24, 22), (25, 22), (26, 22), (26, 23), (26, 24), (27, 24), (27, 25), (28, 25), (29, 25), (30, 25), (30, 26), (31, 26), (31, 27), (32, 27), (33, 27), (33, 28), (34, 28), (35, 28), (35, 29), (36, 29), (36, 30), (36, 31), (37, 31), (37, 32), (37, 33), (38, 33), (38, 34), (38, 35), (38, 36), (39, 36), (39, 37), (39, 38), (39, 39)]


In [ ]:
def findNeighboringOpenCoordsIncludingDiagonals(coords, maze):
    cellRow, cellCol = coords
    neighbors = findNeighboringOpenCoords(coords, maze)
    potentialDiagonalNeighbors = [(cellRow - 1, cellCol - 1), (cellRow - 1, cellCol + 1), (cellRow + 1, cellCol - 1), (cellRow + 1, cellCol + 1)]
    for potentialDiagonalNeighbor in potentialDiagonalNeighbors:
        row, col = potentialDiagonalNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != Cell.OPEN):
            continue
        neighbors.append((row, col))
    return neighbors

visited = {}

def diagonalTravelShortestPathLength(coords, maze):
    cellRow, cellCol = coords 
    shortestPathLength = 0
    expandedCells = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, expandedCellsBySearch = shortestPathSearch(maze, startCoords = (cellRow, cellCol), findNeighborsFunction = findNeighboringOpenCoordsIncludingDiagonals)
        shortestPathLength = len(shortestPath)
        expandedCells = expandedCellsBySearch
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength, expandedCells

if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = diagonalTravelShortestPathLength)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    boardWithPath = copy.deepcopy(maze.board)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")